In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from itertools import product

sys.path.append('./OptimizersArticle')



from data_generator.data_generator import (
    generate_data,
    construct_bounds,
    construct_lp_grid
)

from optimizers.optimizers import (
    ScipyNlpOptimizationModel,
    PyomoNlpOptimizationModel,
    PyomoLpOptimizationModel,
    CvxpyLpOptimizationModel
)
from optimizers.optimization import pricing_optimization


In [3]:
bounds_params = {
    'main_bounds': {
        'lower': 0.9, 'upper': 1.1
    },
    'market_bounds': {
        'lower': 0.85, 'upper': 1.15
    }
}


In [ ]:
seed_grid = list(range(25))
N_grid = [10, 20, 50, 100, 200, 500, 1000]

stats_df_list = []

for N, seed in product(N_grid, seed_grid):
    print('--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--')
    print(N, seed)
    # генерация данных для непрерывной NLP оптимизации
    data_c = generate_data(N, seed)
    data_c = construct_bounds(data_c, bounds_params)
    # генерация данных для MILP оптимизации
    data_d = construct_lp_grid(data_c, bounds_params, 21)
    
    M_cur = sum(data_c['Q'] * (data_c['P'] - data_c['C']))
    P_cur = sum(data_c['Q'] * data_c['P'])

    opt_params = {
        'alpha': 0.0,
        'con_mrg': M_cur,
        'con_equal': True
    }
    
    statuses = []
    times = []
    solvers = []

    if N < 1000:    
        res, t = pricing_optimization(data_c, ScipyNlpOptimizationModel, opt_params, 'slsqp')
        statuses.append('ok' if res['status'] == '0' else res['status'])
        times.append(t)
        solvers.append('slsqp')
        print(f'slsqp finish \t{t}')
    
    if N < 500:
        res, t = pricing_optimization(data_c, ScipyNlpOptimizationModel, opt_params, 'trust-constr')
        statuses.append('ok' if res['status'] == '1' else res['status'])
        times.append(t)
        solvers.append('trust-constr')
        print(f'trust finish \t{t}')

    res, t = pricing_optimization(data_c, PyomoNlpOptimizationModel, opt_params, 'ipopt')
    statuses.append('ok' if res['status'] == 'ok' else res['status'])
    times.append(t)
    solvers.append('ipopt')
    print(f'ipopt finish \t{t}')

    res, t = pricing_optimization(data_d, PyomoLpOptimizationModel, opt_params, 'cbc')
    statuses.append('ok' if res['status'] == 'ok' else res['status'])
    times.append(t)
    solvers.append('cbc')
    print(f'cbc finish \t{t}')

    res, t = pricing_optimization(data_d, PyomoLpOptimizationModel, opt_params, 'glpk')
    statuses.append('ok' if res['status'] == 'ok' else res['status'])
    times.append(t)
    solvers.append('glpk')
    print(f'glpk finish \t{t}')
    
    if N < 2000:
        res, t = pricing_optimization(data_d, CvxpyLpOptimizationModel, opt_params, 'ECOS_BB')
        statuses.append('ok' if res['status'] == 'optimal' else res['status'])
        times.append(t)
        solvers.append('ecos')
        print(f'ecos finish \t{t}')

    stats_df_list.append(pd.DataFrame({
        'N': N,
        'seed': seed,
        'solver': solvers,
        't': times,
        'status': statuses
    }))
    
stats_df = pd.concat(stats_df_list)
stats_df.to_csv('./data/optimizers_stats.csv', header=None)


--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
10 0
slsqp finish 	0.015349149703979492
trust finish 	0.25341200828552246
ipopt finish 	0.04384875297546387
cbc finish 	0.0781562328338623
glpk finish 	0.03818225860595703
ecos finish 	0.017927885055541992
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
10 1
slsqp finish 	0.01718592643737793
trust finish 	0.18254709243774414
ipopt finish 	0.04299116134643555
cbc finish 	0.1481153964996338
glpk finish 	0.042098045349121094
ecos finish 	2.261557102203369
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
10 2
slsqp finish 	0.009814977645874023
trust finish 	0.5899927616119385
ipopt finish 	0.04700303077697754
cbc finish 	0.07006978988647461
glpk finish 	0.03888392448425293
ecos finish 	0.01810479164123535
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
10 3
slsqp finish 	0.011662006378173828
trust finish 	0.2995481491088867
ipopt finish 	0.04971909523010254
cbc finish 	0.0674390792846

cbc finish 	0.08479094505310059
glpk finish 	0.04870867729187012
ecos finish 	4.673716068267822
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
20 6
slsqp finish 	0.013175725936889648
trust finish 	0.41027188301086426
ipopt finish 	0.04558897018432617
cbc finish 	0.12346196174621582
glpk finish 	0.045507192611694336
ecos finish 	2.9935569763183594
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
20 7
slsqp finish 	0.020315885543823242
trust finish 	0.5410878658294678
ipopt finish 	0.04522109031677246
cbc finish 	0.17683005332946777
glpk finish 	0.05386209487915039
ecos finish 	3.809580087661743
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
20 8
slsqp finish 	0.017293214797973633
trust finish 	1.3004460334777832
ipopt finish 	0.05196094512939453
cbc finish 	0.07236409187316895
glpk finish 	0.0438382625579834
ecos finish 	0.022037982940673828
--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--
20 9
slsqp finish 	0.01662302017211914